In [26]:
import folium
import pandas as pd
from folium.plugins import HeatMap
from folium import Icon
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
df = pd.read_csv('../results/CreateLabellingResult.csv')

In [4]:
df

,timestamp,username,handle,tweet_date,content,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,...,location_unique,tokenized_location,address,latitude,longitude,city,latitude_city,longitude_city,initial_label,final_label
0,2024-11-03 19:38:25,Orhan Şevik,orhan_sevik,"Feb 8, 2023",@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
1,2024-11-03 19:38:25,nursena,nurssxx_,"Feb 8, 2023",allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
2,2024-11-03 19:38:25,Elif ŞEKER,55ellllllif,"Feb 8, 2023",replying to \n@haluklevent\n @danlabilic\n and...,replying to \n\n \n and \n_harun\nadıyamanda a...,replying to and harun adıyamanda ali taş...,harun adıyamanda ali taşı mahallesi sokak hic...,harun adıyamanda ali taşı mahallesi sokak hicr...,"['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...",...,Adıyaman Ali Taşı Mahallesi Sokak Hicret Apart...,"['Adıyaman', 'Ali', 'Taşı', 'Mahallesi', 'Soka...",NaN,NaN,NaN,['Adıyaman'],37.789360,38.314110,destek bilgilendirme,destek
3,2024-11-03 19:38:25,Yusuf,yusufaltuns,"Feb 8, 2023",tuğba södekoğlu kovulsun \n@showtv\n #deprem,tuğba södekoğlu kovulsun \n\n #deprem,tuğba södekoğlu kovulsun deprem,tuğba södekoğlu kovulsun deprem,tuğba sodekoğlu kovulsun deprem,"['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
4,2024-11-03 19:38:25,𝐎̈𝐳𝐠𝐮̈𝐫 𝐑𝐚𝐧,dryghtn2,"Feb 8, 2023",arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşımıza ulaşamıyoruz kahramanmaraş elbist...,"['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...",...,Kahramanmaraş Elbistan Pınarbaşı Mahallesi Cad...,"['Kahramanmaraş', 'Elbistan', 'Pınarbaşı', 'Ma...",kahramanmaraş,NaN,NaN,['Kahramanmaraş'],37.783034,36.830655,çok acil acil,acil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2024-11-17 16:28:31,KEREMCGRC,keremcgrci,"Feb 8, 2023",yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol twitterda paylaşılan...,yardım ağı´na destek ol twitterda paylaşılan a...,yardım ağına destek ol twitterda paylaşılan ad...,"['yardım', 'ağına', 'destek', 'ol', 'twitterda...",...,Hataykahramanmaraşhatay,['Hataykahramanmaraşhatay'],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
2098,2024-11-17 16:28:31,aMMo,tatavaypmadvmet,"Feb 8, 2023",teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim emlak bank evleri kıs...,teyi̇tli̇ lütfen yardim emlak evleri kısım blo...,teyi̇tli̇ lütfen yardım emlak evleri kısım blo...,"['teyi̇tli̇', 'lütfen', 'yardım', 'emlak', 'ev...",...,Hatay Antakya Cebrail Mahallesi,"['Hatay', 'Antakya', 'Cebrail', 'Mahallesi']",hatay antakya,36.202547,36.160291,['Hatay'],36.345133,36.074802,çok acil acil,çok acil
2099,2024-11-17 16:28:31,N’Y,nyonyedi,"Feb 8, 2023",05324707903 babalatv\n\nwhastapp hatti tüm böl...,05324707903 babalatv\n\nwhastapp hatti tüm böl...,babalatv whastapp hatti tüm bölgelerdeki yar...,babalatv whastapp hatti tüm bölgelerdeki yardi...,babalatv Whatsapp hattı tüm bölgelerdeki yardı...,"['babalatv', 'Whatsapp', 'hattı', 'tüm', 'bölg...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,çok acil 

In [20]:
label_styles = {
    "çok acil": {"color": "red", "icon": "exclamation-circle"},
    "bilgilendirme": {"color": "blue", "icon": "info-circle"},
    "destek": {"color": "green", "icon": "question-circle"},
    "acil": {"color": "orange", "icon": "circle"}
}

In [ ]:
m = folium.Map(location=[37.3741, 36.7784], zoom_start=7)

for index, row in df.iterrows():
    if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
        label = row["final_label"]
        if label in label_styles:
            marker_color = label_styles[label]["color"]
            marker_icon = label_styles[label]["icon"]
        else:
            marker_color = "gray"  
            marker_icon = "circle"

        tweet_content = row["content"]  
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f'<strong>Tweet Content:</strong><br>{tweet_content}',  
            icon=Icon(color=marker_color, icon=marker_icon)
        ).add_to(m)
m.save("../web/address_map_new.html")

# HEATMAP

In [37]:
geolocator = Nominatim(user_agent="geoapi")
def get_coordinates(address):
    if not address or pd.isna(address):
        return None, None
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [ ]:
non_empty_cities = df['city'].dropna()
non_empty_cities = non_empty_cities[non_empty_cities.str.strip() != '']
city_exploded = non_empty_cities.str.replace(r"[\[\]']+", '', regex=True)  
city_exploded = city_exploded.str.split(',').explode().str.strip()
city_count = city_exploded.value_counts()
city_count_df = city_count.reset_index()
city_count_df.columns = ['City', 'Count']
city_count_df = city_count_df[city_count_df['City'] != '']
city_count_df.head(20)

,City,Count
1,Hatay,423
2,Kahramanmaraş,208
3,Adıyaman,165
4,Gaziantep,87
5,Malatya,64
6,Adana,51
7,Diyarbakır,29
8,İstanbul,23
9,Şanlıurfa,20
10,Osmaniye,19


In [48]:
city_count_df[['Latitude', 'Longitude']] = city_count_df['City'].apply(lambda x: pd.Series(get_coordinates(x)))

In [49]:
city_count_df

,City,Count,Latitude,Longitude
1,Hatay,423,36.345133,36.074802
2,Kahramanmaraş,208,37.783034,36.830655
3,Adıyaman,165,37.789360,38.314110
4,Gaziantep,87,37.062832,37.379262
5,Malatya,64,38.348715,38.319067
6,Adana,51,36.986360,35.325286
7,Diyarbakır,29,37.916222,40.236354
8,İstanbul,23,41.076602,29.052495
9,Şanlıurfa,20,37.259520,39.040817
10,Osmaniye,19,37.251788,36.299350


In [56]:
m = folium.Map(location=[41.008238, 28.978359], zoom_start=6)
heat_data = [[row['Latitude'], row['Longitude'], row['Count']] for index, row in city_count_df.iterrows()]
HeatMap(heat_data, 
        min_opacity=0.4,  
        max_opacity=0.9, 
        radius=50,  
        blur=30,  
        gradient={  
            0.2: 'blue',   
            0.5: 'lime',   
            0.7: 'yellow',  
            0.8: 'orange',
            1.0: 'red'      
        }).add_to(m)

for index, row in city_count_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=10,  
        color='black',
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['City']}: {row['Count']}",
        tooltip=f"{row['City']} - {row['Count']} people"
    ).add_to(m)

m.save('../web/heatmap.html')